## Install Lib API

In [ ]:
! pip install https://dnaink.jfrog.io/artifactory/dna-ink-pypi/model-fkeywords/0.1.0/model_fkeywords-0.1.0-py3-none-any.whl

In [ ]:
! python -m spacy download pt_core_news_sm

## Import libs

In [1]:
import pandas as pd
from api_model.nlsuper import NlExtractorProcess
from api_model.nlvisualization import NlVisualization

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_1949/1571986624.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Variables

##### Description of process
        whats_process = 'complete'
            return: process all pipeline
        whats_process = 'partial'
            return: findkeywords and process bigrams
        whats_process = 'only_keywords'
            return: findkeywords   

In [2]:
filename = 'results_100_call'
prefix = 'csv'
prefix_sep = ','
column_text = 'TEXT'
whats_process = 'complete'
#whats_process = 'partial'
#whats_process = 'only_keywords'
id_database = 'ID'
type_find = 'aproximado' # aproximado # fixo
activate_stopwords = 'sim' # não
encoding = 'UTF-8' #UTF-8

In [3]:
text_finds = {
    'negatividade': ['a desejar', 'abrir uma reclamacao', 'absurd', 'aguento mais', 'boceta', 'bosta', 'brigaram', 'buceta', 'burocra', 'cansad', 'caralho', 'conflito', 'constrang', 'decepcao', 'decepcionad', 'quanta demora', 'que demora', 'tentando resolver', 'descontent', 'descrenca', 'descrente', 'desrespeit', 'fdp', 'filha da puta', 'filha de uma puta', 'filho da puta', 'filho de uma puta', 'frustracao', 'frustrad', 'humilhacao', 'humilhad', 'ignorad', 'ignoram', 'incompet', 'inferno', 'informa nada', 'injuria', 'ironia', 'ironic', 'irreponsaveis', 'irresponsabilidade', 'irresponsavel', 'ma vontade', 'mal a pior', 'mal atendid', 'mal educad', 'mal respondem', 'mal tratad', 'mau atendid', 'mau educad', 'mau respondem', 'mau tratad', 'merda', 'nao podemos arcar', 'nao posso arcar', 'nao sabe', 'ninguem resolve', 'ofenderam', 'ofendid', 'orrivel', 'pelo amor de deus', 'pessimo', 'pessimu', 'porra', 'poxa', 'pqp', 'puta que pariu', 'reclamacao', 'reclamar', 'ridicul', 'ruim', 'sabe nada', 'sem educacao', 'ta dificil', 'trata mal', 'trata mau', 'tratou mal', 'tratou mau', 'triste', 'vergonh', 'vagabund', 'raiva', 'detest', 'nao quero mais', 'odeio', 'safad', 'deus me livre', 'vergonh'],
    'cancelamento': ["podem cancelar", "quero cancelar", "vou cancelar", "ja pedi o cancelamento", "ja solicitei o cancelamento", "pedir o cancelamento", "exijo o cancelamento", "vamos de cancelamento", "eu fiz o cancelamento", "tentando o cancelamento", "solicitacao de cancelamento", "quero fazer o cancelamento", "quero o cancelamento", "realizar o cancelamento", "solicitar o cancelamento", "ja esta cancelado", "cancelamento do servico", "confirmar meu cancelamento", "setor de cancelamento", "area de cancelamento", "clico em cancelamento", "pedido de cancelamento", "dificil cancelar", "cancelamento do plano", "cancelamento do pacote", "cancelamento do servico", "gostaria do cancelamento", "gostaria de cancelar", "tentando cancelar"],
    'rechamada': ["central de atendimento", "ja liguei na central", "liguei algumas vezes", "liguei e me informaram", "liguei varias vezes", "liguei ontem", "liguei hoje", "liguei agora no 10621", "liguei no 10621", "quando ligo na central", "falei com atendente", "liguei na claro", "liguei diversas vezes", "ja liguei varias vezes"],
    'satisfacao': ['adorei', 'adoro', 'ageis', 'agil', 'agilidade', 'amei', 'atendimento perfeito', 'bem atendid', 'carisma', 'diferenciad', 'educad', 'eficacia', 'eficas', 'eficaz', 'elogiar', 'elogio', 'excelente atendimento', 'feliz', 'gostei', 'impecavel', 'me ajudou', 'merito', 'motivacao', 'motivad', 'muito 10', 'muito bom', 'muito dez', 'nota 10', 'nota dez', 'otimo atendimento', 'parabenizar', 'parabens', 'pela ajuda', 'personalizad', 'preparad', 'prestativ', 'profissional', 'qualidade', 'rapidez', 'resolveu', 'satisfacao', 'satisfatori', 'satisfeit', 'solucionou', 'ate que enfim', 'ate que em fim'],
    'concorrencia': ["clarobr", "claro", "operadora claro", "migrar para a claro", "mudar para a claro", "contratar a claro", "assinar a claro", "to indo para a claro", "timbrasil", "tim", "operadora tim", "migrar para a tim", "mudar para a tim", "contratar a tim", "assinar a tim", "to indo para a tim", "tim_ajuda", "oi_oficial", "operadora oi", "migrar para a oi", "mudar para a oi", "contratar a oi", "assinar a oi", "to indo para a oi"],
    'contencao': ['desculpa', 'desculpe', 'sinto muito', 'desculpe a demora', 'lamento', 'lamentamos', 'perdoa', 'perdao', 'pelo ocorrido', 'o ocorrido', 'fique tranquil'],
    'ocorrencia': ['chamado', 'ocorrencia', 'registrada com sucesso'],
    'direcionamento': ['central de atedimento', 'ligar na central', 'somente na central', 'sodexo.com', 'ligue na central'],
    'odc': ["ponho na justica","vou entrar na justica","vou entrar com uma acao","botar na justica","vou ter que entrar na justica","pequenas causas","por na justica","pra mim entrar na justica","eu vou entrar na justica","vou levar pro juiz de pequenas causas","ainda vou pra justica","vou entrar na justica contra voces","voces querem que bote na justica","eu vou botar na justica","vou recorrer nos pequenas causas","eu entro na justica","orgaos competentes", "anatelgovbr", "anatel", "medidas cabiveis", "consumidor_gov", "consumidor.gov",  "reclameaqui", "reclame aqui", "procurar meus direitos", "tomando devidas providencias", "vou reclamar na", "vou reclamar no", "justica", "procon", "procom", "vou processar", "vou entrar com um processo"],
    'alto_atrito':['desumano', 'estou chateado', 'estou triste', 'falta de respeito', 'horrivel', 'irritado', 'nao aguento mais', 'nao quero saber da claro', 'palhacada', 'sacanagem', 'voces estao errados', 'absurdo', 'brincadeira', 'pessimo', 'porcaria'],
    'nao_pertube':["Não perturbe", "não me perturbe", "parem de me ligar", "não receber ligações", "NãoMePerturbe", "não quero receber ligações", "diversas ligações", "vivem me ligando", "não param de ligar", "perturbação", "Ligações o dia todo", "Ligações indesejadas", "Tirem meu número do cadastro", "pararem de me ligar", "bloqueio das chamadas", "tirem meu numero da lista", "chamadas por dia", "ligações por dia"],
    'nao_resolvido':["não resolveram meu problema", "não resolvem meu problema", "não resolveram meu pobrema", "ninguém resolve meu problema","ninguém resolveu o meu problema", "ninguém resolve nada", "ninguém nunca resolve", "nunca conseguem resolver", "não consegue resolver", "não sabe qual setor resolve meu problema", "não sabia dar informação sobre meu caso", "nao sabia dar informacao sobre meu caso", "não domina o procedimento", "nao domina o procedimento", "não resolveu meu problema ainda", "não resolveu meus problemas ainda", "nenhum atendente conseguia solucionar meu problema", "nenhum atendente conseguia resolver meu poblema", "não resolveram o problema", "nao resolveram o problema", "não resolvi meu poblema", "eu continuo com o mesmo problema", "eu continuo com o mesmo poblema", "eu permaneço com o mesmo problema", "eu permaneco com o mesmo poblema", "não foi me passado por isso estou aqui", "nunca passam por isso estou aqui", "permaneço sem solução", "permaneco sem solução", "meu problema continua", "esse problema não tem fim", "o problema persiste sem solução", "o problema persiste sem solucao", "não resolveu meu problema", "não resolveu meus problemas”, “nenhum atendente conseguiu solucionar meu problema”, “nenhum atendente conseguiu solucionar meu poblema", "nenhum atendente conseguia solucionar meu problema", "não solucionei meu problema", "não solucionaram o poblema", "não solucionaram nada", "não resolvem os problemas", "nada de conseguir resolver", "não resolução de problema”, “nao resolucao de problema”, “vários atendimentos não resolveram”, “diversos atendimentos não resolvem”, “segue sem resolver meu caso”, “permaneço na mesma sem resolver”, “nunca resolve”, “nunca resolveram”, “esse contato não me serviu”, “nenhum canal resolve”, “ninguém resolve minha reclamação”, “ninguem resolve minha reclamacao”, “ninguém está nem ai pro meu problema”,” tudo errado","ninguém resolve nada", "ninguém resolve meu problema"]
          }

In [4]:
additional_stop_words = ['porque','bom','dia','tres','três','alo','alô',
                         'nan','nome','completo','cpf', 'pra', 'mim', 'atendimento','vivo','meia']

In [5]:
interlocutor = {'INTERLOCUTOR': ['SPEAKER_00', 'SPEAKER_01']}
response_time = ''
format_data = '%d/%m/%Y %H:%M:%S|%d/%m/%Y %H:%M|%Y-%m-%d %H:%M:%S|%d-%m-%Y %H:%M|%d%m%Y %H:%M:%S|%d%b%Y:%H:%M:%S' # 03MAR2022:12:01:33

In [ ]:
df = NlExtractorProcess.call_process(filename, prefix, prefix_sep,\
                                     column_text, whats_process,\
                                     text_finds, id_database, type_find,\
                                     additional_stop_words, activate_stopwords,\
                                    interlocutor, response_time, format_data, encoding)

[07/19 19:05:53] - [Info]:  Load CSV
[07/19 19:05:53] - [Info]:  read file /opt/dna/find-keywords/datalake/csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597 entries, 0 to 8596
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            8597 non-null   object 
 1   PROB_1        8597 non-null   float64
 2   PROB_2        8597 non-null   float64
 3   INTERLOCUTOR  8597 non-null   object 
 4   TEXT          7079 non-null   object 
dtypes: float64(2), object(3)
memory usage: 335.9+ KB
[07/19 19:05:53] - [Debug]:  Schema of dataframe is None
[07/19 19:05:53] - [Info]:  Normalize Datas Values
[07/19 19:05:53] - [Info]:  convert column_text column to string type
[07/19 19:05:53] - [Info]:  put column_text in lower case
[07/19 19:05:53] - [Info]:  normalize id column
[07/19 19:05:53] - [Info]:  create date column timestamp
[07/19 19:05:53] - [Debug]:  dataformat: 0    2022-07-19 22:05:53
1    2022-07-19 22:05

/opt/dna/find-keywords/api_model/utils/functions.py:152: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df[response_time] =pd.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


[07/19 19:06:29] - [Info]:  result after of process stop words 
 0    tendi  
1    técnico
2           
3    falo   
4    miguel 
Name: TEXT, dtype: object
[07/19 19:06:29] - [Info]:  Start Text Mining
[07/19 19:06:29] - [Info]:  remove special characters and pontuation of column_text
[07/19 19:06:30] - [Info]:  tranform text in text lemma
[07/19 19:08:07] - [Info]:  Start Word Search
[07/19 19:08:07] - [Info]:  collect words and find in column_text
[07/19 19:08:07] - [Debug]:  dict: {'negatividade': ['a desejar', 'abrir uma reclamacao', 'absurd', 'aguento mais', 'boceta', 'bosta', 'brigaram', 'buceta', 'burocra', 'cansad', 'caralho', 'conflito', 'constrang', 'decepcao', 'decepcionad', 'quanta demora', 'que demora', 'tentando resolver', 'descontent', 'descrenca', 'descrente', 'desrespeit', 'fdp', 'filha da puta', 'filha de uma puta', 'filho da puta', 'filho de uma puta', 'frustracao', 'frustrad', 'humilhacao', 'humilhad', 'ignorad', 'ignoram', 'incompet', 'inferno', 'informa nada', 'in

22/07/19 22:09:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[07/19 19:10:00] - [Info]:  Finishing Process
[07/19 19:10:00] - [Info]:  created message order
Generating `message_order` column
root
 |-- issue_id: string (nullable = true)
 |-- PROB_1: string (nullable = true)
 |-- PROB_2: string (nullable = true)
 |-- message_author: string (nullable = true)
 |-- message_content: string (nullable = true)
 |-- message_time: string (nullable = true)
 |-- negatividade_findint: string (nullable = true)
 |-- cancelamento_findint: string (nullable = true)
 |-- rechamada_findint: string (nullable = true)
 |-- satisfacao_findint: string (nullable = true)
 |-- concorrencia_findint: string (nullable = true)
 |-- contencao_findint: string (nullable = true)
 |-- ocorrencia_findint: string (nullable = true)
 |-- direcionamento_findint: string (nullable = true)
 |-- odc_findint: string (nullable = true)
 |-- alto_atrito_findint: string (nullable = true)
 |-- nao_pertube_findint: string (nullable = true)
 |-- nao_resolvido_findint: string (nullable = true)
 |-- n

[07/19 19:10:10] - [Info]:  numbers of rows agrouped 5179
[07/19 19:10:10] - [Info]:  Generating wordcloud columns for "all_messages" with prefix "countent"


22/07/19 22:11:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
filename = 'results_100_call'
column_filter = '' # deixar esse campo vazio caso queira ver tudo
column_text = 'all_messages'
whats_process = 'trigram' # trigram or bigram

In [ ]:
NlVisualization.wordCloud_Topics(filename,column_filter,column_text,whats_process)

In [ ]:
NlVisualization.plot_10_most_common_words(filename,column_filter,column_text)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (46, 9)
NlVisualization.pareto_plot(filename, x='countent_bigram', y='count', title='Alias Pareto Distribution', limite=20)

In [ ]:
NlVisualization.tsne_plot(filename, column_text, 'pagamento', 40, n_iter=300, vector_size=300, min_count=20, window=30)

In [ ]:
clst, optimal_k, df = NlVisualization.clustering_model(filename, column_text, model='kmeans', plot=True, max_k=20)

In [ ]:
df.head()